In [25]:
import pickle
import spacy
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [26]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "textcat", "ner"])

In [27]:
df = pd.read_csv('data/df_white.csv', index_col = 0)

In [28]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery', 'type'],
      dtype='object')

In [29]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,type
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,white
2,US,"Tart and snappy, the flavors of lime flesh and...",Pinot Gris,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,white
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,white
7,France,This dry and restrained wine offers spice in p...,Gewürztraminer,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,white
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,white


In [30]:
def custom_tokenizer(text):
    '''
    used to filter out unwanted words, punctuation, and so on
    '''
    tokens = []
    for t in nlp(text):
        if not(len(t) < 2 or t.is_stop or t.like_num or 
               t.is_punct or not t.is_alpha):
            tokens.append(t.lemma_)
    return tokens 

In [31]:
corpus = df['description']

In [32]:
corpus

0         Aromas include tropical fruit, broom, brimston...
2         Tart and snappy, the flavors of lime flesh and...
3         Pineapple rind, lemon pith and orange blossom ...
7         This dry and restrained wine offers spice in p...
8         Savory dried thyme notes accent sunnier flavor...
                                ...                        
129965    While it's rich, this beautiful dry wine also ...
129966    Notes of honeysuckle and cantaloupe sweeten th...
129968    Well-drained gravel soil gives this wine its c...
129969    A dry style of Pinot Gris, this is crisp with ...
129970    Big, rich and off-dry, this is powered by inte...
Name: description, Length: 32945, dtype: object

In [33]:
y = df['variety']

In [34]:
y.value_counts()

Chardonnay                    11753
Riesling                       5189
Sauvignon Blanc                4967
White Blend                    2360
Pinot Gris                     1455
Grüner Veltliner               1345
Portuguese White               1159
Bordeaux-style White Blend     1066
Pinot Grigio                   1052
Gewürztraminer                 1012
Viognier                        996
Chenin Blanc                    591
Name: variety, dtype: int64

In [35]:
bow = CountVectorizer(tokenizer=custom_tokenizer, 
                      ngram_range=(1, 1), 
                      min_df=0.01, 
                      max_df=0.99)

In [36]:
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, 
                                                              test_size=0.4, 
                                                              train_size=0.6)

In [37]:
X_train = bow.fit_transform(corpus_train)

In [38]:
X_train.shape

(19767, 364)

In [39]:
X_test = bow.transform(corpus_test)

In [40]:
m_rf = Pipeline([
    ('TfIdf', TfidfTransformer()),
    ('RFC', RandomForestClassifier())
])

In [41]:
cross_val_score(m_rf, X_train, y_train, scoring='accuracy', n_jobs=4, cv=4).mean()

0.6986392282023793

In [42]:
m_rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('TfIdf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('RFC',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=

In [43]:
m_rf.predict(X_test)

array(['Chardonnay', 'Chardonnay', 'White Blend', ..., 'Chardonnay',
       'White Blend', 'Chardonnay'], dtype=object)

In [44]:
metrics.accuracy_score(y_test, m_rf.predict(X_test))

0.6996509333738048

In [47]:
m_rf.predict(bow.transform(['cherry flavor nice oak accent']))

array(['Chardonnay'], dtype=object)

In [48]:
m_rf.predict_proba(bow.transform(['cherry flavor nice oak accent'])).max()

0.89

In [49]:
with open('m_white_rf.p', 'wb') as f:
    pickle.dump(m_rf, f)

In [50]:
with open('bow_white_rf.p', 'wb') as f:
    pickle.dump(bow, f)